# Test of Sampler

Imports.

In [1]:
%matplotlib widget

import datetime
import itertools
import kiruna
import matplotlib.pyplot as plt
import numpy
import pandas
import pickle
import torch

## Create a scene

In [2]:
scene_file_name = r"D:\analog\mixedreality.Platform.EyeTracking.Tools\kiruna\kiruna\scene\atlas_1.2_scene\atlas_1.2_scene.json"
scene_sampler_file_name = r"D:\analog\mixedreality.Platform.EyeTracking.Tools\kiruna\kiruna\sampler\default_sampler\default_scene_sampler.json"

In [3]:
# Scene and sampler.
num_scene_samples = 5
et_scene = kiruna.scene.SceneModel(parameter_file_name=scene_file_name)
scene_sampler = kiruna.sampler.SceneSampler(et_scene, num_scene_samples)

# Noise.
noise_std_pix = torch.tensor([0.0, 0.2, 0.5, 1.0, 3.0])
noise_std_radius = torch.tensor([0.0, 0.3, 0.7])

# Gaze grid.
H, V = 2, 2
gaze_range_deg = et_scene.device.display_fov.tolist()
h_angles_deg = torch.linspace(0.0, gaze_range_deg[0], H) / 2
v_angles_deg = torch.linspace(0.0, gaze_range_deg[1], V) / 2

# Experiment repetitions.
num_experiment_repetitions = 20 #50

subsystem_index = 0
camera_index = 0

torch.manual_seed(0)

In [4]:
et_scene.device.subsystems[1].children

## Test Limbus-Only Eye Tracking

In [28]:
header = [
    "Scene",
    "Horiz. angle [deg]",
    "Vert. angle [deg]",
    "Std of noise in points [pix]",
    "Std of noise in radius [mm]",
    "Repetition",
    "Error x [mm]",
    "Error y [mm]",
    "Error z [mm]",
    "Gaze error [deg]"
]

data = []
for i, scene_sample in enumerate(scene_sampler.generate_samples()):

    print("Scene {0:d}".format(i))
    eye = scene_sample.user.eyes[subsystem_index]

    limbus = eye.limbus
    
    # Attach the limbus to a limbus plane.
    center_inEye, normal_inEye = limbus.get_center_and_normal_inParent()
    limbus_plane = \
        kiruna.primitive.Plane.create_from_origin_and_normal_inParent(
            eye, center_inEye, normal_inEye
        )
    # Currently, the pose graph is
    #
    # Eye
    #  | \_
    #  |   \_
    #  |     \
    #  V      V
    # plane limbus.
    #
    # We want
    #
    # Eye ---> plane ---> limbus.
    limbus_plane.add_child(limbus)

    camera = \
        scene_sample.device.subsystems[subsystem_index].cameras[camera_index]
    
    for v in range(V):
        vertical_angle = v_angles_deg[v]
        for h in range(H):
            horizontal_angle = h_angles_deg[h]
            
            # Rotate the eye.
            gaze_angle_deg = torch.stack((horizontal_angle, vertical_angle))
            eye.rotate_from_gaze_angles_inParent(gaze_angle_deg)
            
            # Get the true limbus parameters.
            limbus = eye.limbus
            radius = limbus.radius
            
            center_inCamera, \
                    normal_inCamera = \
                        limbus.get_center_and_normal_inOther(camera)

            # Sample noise-free points in limbus image.
            limbus_image = camera.project_ellipse_toImagePlane(limbus)
            limbus_points_inImagePlane, _ = limbus_image.get_points_inParent()

            # Move points to pixels so we can add noise.
            limbus_points_inPixels = []
            for point_inImage in limbus_points_inImagePlane.T:
                point_inPixels = camera.get_point_inPixels(point_inImage[:2])
                limbus_points_inPixels = \
                    limbus_points_inPixels + [point_inPixels, ]

            # Add noise and move points back to the image plane.
            combo = \
                itertools.product(
                    noise_std_pix,
                    noise_std_radius,
                    range(num_experiment_repetitions)
                )
            for std_pix, std_radius, j in combo:
                noisy_points_inImagePlane = []
                for point_inPixels in limbus_points_inPixels:
                    # Noise.
                    point_inPixels = \
                        point_inPixels + \
                            torch.randn_like(point_inPixels) * std_pix

                    # Move back to image plane.
                    noisy_inImagePlane = \
                        camera.get_point_inImagePlane(point_inPixels)

                    # Save results.
                    noisy_points_inImagePlane = \
                        noisy_points_inImagePlane + [noisy_inImagePlane, ]

                # Fit an ellipse to the noisy points in the image plane.
                ellipse_fitter = \
                    kiruna.optimization.EllipseFitting(
                        camera.image_plane, noisy_points_inImagePlane
                    )
                # Use ground truth to initialize.
                gt_center_inParent, \
                    _ = limbus_image.get_center_and_normal_inParent()
                gt_x_extreme_inSelf = torch.tensor([1.0, 0.0, 0.0])
                T_toParent_fromSelf = \
                    limbus_image.get_transform_toParent_fromSelf()
                gt_x_extreme_inParent = \
                    T_toParent_fromSelf.transform(gt_x_extreme_inSelf)
                delta = gt_x_extreme_inParent - gt_center_inParent
                # This is just an approximation.
                gt_angle_rad = torch.atan2(delta[1], delta[0])
                gt_angle_deg = kiruna.core.rad_to_deg(gt_angle_rad)
                ellipse_fitter.set_initial_values(
                    gt_center_inParent,
                    gt_angle_deg,
                    limbus_image.x_radius,
                    limbus_image.y_radius
                )
                try:
                    noisy_limbus_in_image = \
                        ellipse_fitter.fit(options={"disp": True})
                except:
                    print("Failed fit!")
                    print("\tScene {:d}".format(i))
                    print("\tHoriz. angle: {:.2f}".format(horizontal_angle))
                    print("\tVert. angle: {:.2f}".format(vertical_angle))
                    print("\tStd points: {:.2f}".format(std_pix))
                    print("\tStd radius: {:.2f}".format(std_radius))
                    print("\tRepetition: {:d}".format(j))
                    continue

                # Forward project the noisy limbus image into 3D.
                noisy_radius = radius + torch.randn_like(radius) * std_radius
                sol_pos, \
                    sol_neg = \
                    camera.forward_project_to_circle_from_ellipse_inImagePlane(
                        noisy_limbus_in_image, noisy_radius
                    )
                # Manually solve the ambiguity.
                difference_pos = sol_pos[0] - center_inCamera
                difference_neg = sol_neg[0] - center_inCamera
                dist_square_pos = difference_pos @ difference_pos
                dist_square_neg = difference_neg @ difference_neg
                if dist_square_pos < dist_square_neg:
                    center_inCamera_ = sol_pos[0]
                    normal_inCamera_ = sol_pos[1]
                else:
                    center_inCamera_ = sol_neg[0]
                    normal_inCamera_ = sol_neg[1]

                error_center_inCamera = \
                    (center_inCamera_ - center_inCamera).tolist()
                error_normal_deg = \
                    kiruna.core.rad_to_deg(
                        torch.asin(
                            torch.linalg.norm(
                                torch.cross(normal_inCamera, normal_inCamera_)
                            )
                        )
                    )

                row = [
                    i,                      # Scene
                    horizontal_angle,       # Horiz. angle
                    vertical_angle,         # Vert. angle
                    std_pix.item(),         # Std of noise in points 
                    std_radius.item(),      # Std of noise in radius
                    j,                      # Repetition
                    *error_center_inCamera, # Error x, y, z
                    error_normal_deg.item() # Gaze error
                ]
                data = data + [row, ]

            # Unrotate the eye.
            eye.unrotate_from_gaze_angles_inParent(gaze_angle_deg)

            # Remove the projection of the limbus so as not to bloat the pose
            # graph.
            limbus_image.parent.remove_child(limbus_image)

df = pandas.DataFrame(data, columns=header)

now = datetime.datetime.now()
prefix = now.strftime("%Y-%m-%d @ %H-%M-%S.%f")

full_prefix = "limbus_only " + prefix

df_name = full_prefix + " data_frame.pkl"
#with open(df_name, 'wb') as file_stream:
#    pickle.dump(df, file_stream)

Scene 0
Failed fit!
	Scene 0
	Horiz. angle: 0.00
	Vert. angle: 0.00
	Std points: 0.00
	Std radius: 0.00
	Repetition: 0
Failed fit!
	Scene 0
	Horiz. angle: 0.00
	Vert. angle: 0.00
	Std points: 0.00
	Std radius: 0.00
	Repetition: 1
Failed fit!
	Scene 0
	Horiz. angle: 0.00
	Vert. angle: 0.00
	Std points: 0.00
	Std radius: 0.00
	Repetition: 2
Failed fit!
	Scene 0
	Horiz. angle: 0.00
	Vert. angle: 0.00
	Std points: 0.00
	Std radius: 0.00
	Repetition: 3
Failed fit!
	Scene 0
	Horiz. angle: 0.00
	Vert. angle: 0.00
	Std points: 0.00
	Std radius: 0.00
	Repetition: 4
Failed fit!
	Scene 0
	Horiz. angle: 0.00
	Vert. angle: 0.00
	Std points: 0.00
	Std radius: 0.00
	Repetition: 5
Failed fit!
	Scene 0
	Horiz. angle: 0.00
	Vert. angle: 0.00
	Std points: 0.00
	Std radius: 0.00
	Repetition: 6
Failed fit!
	Scene 0
	Horiz. angle: 0.00
	Vert. angle: 0.00
	Std points: 0.00
	Std radius: 0.00
	Repetition: 7
Failed fit!
	Scene 0
	Horiz. angle: 0.00
	Vert. angle: 0.00
	Std points: 0.00
	Std radius: 0.00
	Repetit

AttributeError: 'Plane' object has no attribute 'remvove_child'

## Data Analysis

In [ ]:
#pickle_file_name = df_name
#with open(pickle_file_name, 'rb') as file_stream:
#    df = pickle.load(file_stream)

grouped_by_noise = \
    df.groupby(
        by=["Std of noise in points [pix]", "Std of noise in radius [mm]"]
    )

all_gaze_xy_table = numpy.ndarray((3, 5))
all_gaze_xyz_table = numpy.ndarray((3, 5))

nominal_gaze_xy_table = numpy.ndarray((3, 5))
nominal_gaze_xyz_table = numpy.ndarray((3, 5))

for n, group in enumerate(grouped_by_noise):
    i = n % len(noise_std_radius)
    j = n // len(noise_std_radius)
    
    error_in_xy = \
        group[1][["Error x [mm]", "Error y [mm]"]].to_numpy()
    rms = numpy.sqrt((error_in_xy**2).sum(axis=1).mean())
    all_gaze_xy_table[i, j] = rms
    
    error_in_xyz = \
        group[1][["Error x [mm]", "Error y [mm]", "Error z [mm]"]].to_numpy()
    rms = numpy.sqrt((error_in_xyz**2).sum(axis=1).mean())
    all_gaze_xyz_table[i, j] = rms
    
    # Statistics at nominal gaze
    nominal_group = \
        group[1][
            (group[1]["Horiz. angle [deg]"] == torch.tensor(0.)) &
            (group[1]["Vert. angle [deg]"] == torch.tensor(0.))
        ]
    error_in_xy = \
        nominal_group[["Error x [mm]", "Error y [mm]"]].to_numpy()
    rms = numpy.sqrt((error_in_xy**2).sum(axis=1).mean())
    nominal_gaze_xy_table[i, j] = rms
    
    error_in_xyz = \
        nominal_group[["Error x [mm]", "Error y [mm]",
                       "Error z [mm]"]].to_numpy()
    rms = numpy.sqrt((error_in_xyz**2).sum(axis=1).mean())
    nominal_gaze_xyz_table[i, j] = rms
    
pairs = \
    {
        "RMS error in x and y over all gazes": all_gaze_xy_table,
        "RMS error in x, y, and z over all gazes": all_gaze_xyz_table,
        "RMS error in x and y for nominal gaze": nominal_gaze_xy_table,
        "RMS error in x, y, and z for nominal gaze": nominal_gaze_xyz_table,
    }
for key in pairs:
    print(key)
    header = r"$\sigma_{\rm radius}$ [mm] | "
    for sig_pix in noise_std_pix:
        header = header + "{:.1f} | ".format(sig_pix)
    print(header)
    for i in range(len(noise_std_radius)):
        row = "{:.1f} | ".format(noise_std_radius[i])
        for j in range(len(noise_std_pix)):
            row = row + "{:.2f} | ".format(pairs[key][i, j])
        print(row)
    print("*" * 20)

RMS error in x and y over all gazes
$\sigma_{\rm radius}$ [mm] | 0.0 | 0.2 | 0.5 | 1.0 | 3.0 | 
0.0 | 0.00 | 0.06 | 0.08 | 0.14 | 0.36 | 
0.3 | 0.27 | 0.30 | 0.31 | 0.33 | 0.42 | 
0.7 | 0.72 | 0.71 | 0.62 | 0.64 | 0.86 | 
********************
RMS error in x, y, and z over all gazes
$\sigma_{\rm radius}$ [mm] | 0.0 | 0.2 | 0.5 | 1.0 | 3.0 | 
0.0 | 0.00 | 0.23 | 0.39 | 0.70 | 1.96 | 
0.3 | 2.08 | 2.25 | 2.33 | 2.61 | 2.64 | 
0.7 | 5.51 | 5.49 | 5.00 | 4.99 | 6.15 | 
********************
RMS error in x and y for nominal gaze
$\sigma_{\rm radius}$ [mm] | 0.0 | 0.2 | 0.5 | 1.0 | 3.0 | 
0.0 | 0.00 | 0.04 | 0.07 | 0.12 | 0.38 | 
0.3 | 0.14 | 0.16 | 0.21 | 0.21 | 0.42 | 
0.7 | 0.39 | 0.40 | 0.36 | 0.38 | 0.51 | 
********************
RMS error in x, y, and z for nominal gaze
$\sigma_{\rm radius}$ [mm] | 0.0 | 0.2 | 0.5 | 1.0 | 3.0 | 
0.0 | 0.00 | 0.15 | 0.42 | 0.65 | 1.56 | 
0.3 | 1.86 | 2.07 | 2.38 | 2.90 | 2.79 | 
0.7 | 5.66 | 5.65 | 4.98 | 5.11 | 5.72 | 
********************


In [ ]:
q = 0.80

all_gaze_xy_table = numpy.ndarray((3, 5))
all_gaze_xyz_table = numpy.ndarray((3, 5))

nominal_gaze_xy_table = numpy.ndarray((3, 5))
nominal_gaze_xyz_table = numpy.ndarray((3, 5))

for n, group in enumerate(grouped_by_noise):
    i = n % len(noise_std_radius)
    j = n // len(noise_std_radius)
    
    # Quantitative:
    error_in_xy = \
        group[1][["Error x [mm]", "Error y [mm]"]].to_numpy()
    rms = numpy.quantile(numpy.sqrt((error_in_xy**2).sum(axis=1)), q)
    all_gaze_xy_table[i, j] = rms

    error_in_xyz = \
        group[1][["Error x [mm]", "Error y [mm]", "Error z [mm]"]].to_numpy()
    rms = numpy.quantile(numpy.sqrt((error_in_xyz**2).sum(axis=1)), q)
    all_gaze_xyz_table[i, j] = rms

    # Statistics at nominal gaze
    nominal_group = \
        group[1][
            (group[1]["Horiz. angle [deg]"] == torch.tensor(0.)) &
            (group[1]["Vert. angle [deg]"] == torch.tensor(0.))
        ]
    error_in_xy = \
        nominal_group[["Error x [mm]", "Error y [mm]"]].to_numpy()
    rms = numpy.quantile(numpy.sqrt((error_in_xy**2).sum(axis=1)), q)
    nominal_gaze_xy_table[i, j] = rms

    error_in_xyz = \
        nominal_group[["Error x [mm]", "Error y [mm]",
                       "Error z [mm]"]].to_numpy()
    rms = numpy.quantile(numpy.sqrt((error_in_xyz**2).sum(axis=1)), q)
    nominal_gaze_xyz_table[i, j] = rms

pairs = \
    {
        "80th-quantile in x and y over all gazes": all_gaze_xy_table,
        "80th-quantile error in x, y, and z over all gazes": all_gaze_xyz_table,
        "80th-quantile error in x and y for nominal gaze": nominal_gaze_xy_table,
        "80th-quantile error in x, y, and z for nominal gaze": nominal_gaze_xyz_table,
    }
for key in pairs:
    print(key)
    header = r"$\sigma_{\rm radius}$ [mm] | "
    for sig_pix in noise_std_pix:
        header = header + "{:.1f} | ".format(sig_pix)
    print(header)
    for i in range(len(noise_std_radius)):
        row = "{:.1f} | ".format(noise_std_radius[i])
        for j in range(len(noise_std_pix)):
            row = row + "{:.2f} | ".format(pairs[key][i, j])
        print(row)
    print("*" * 20)

0.00018915793392666657
2.228236303962003
7.646697034855064
0.11850145449854395
2.16717297010353
7.423971474544526
0.4424881668978882
2.951085353641523
6.5964087520539145
0.8673722956813147
3.4787814355139313
6.485886270779949
2.1983027348931965
3.419366304816635
6.6711747244441755
80th-quantile in x and y over all gazes
$\sigma_{\rm radius}$ [mm] | 0.0 | 0.2 | 0.5 | 1.0 | 3.0 | 
0.0 | 0.00 | 0.07 | 0.10 | 0.17 | 0.44 | 
0.3 | 0.32 | 0.36 | 0.36 | 0.39 | 0.51 | 
0.7 | 0.87 | 0.87 | 0.84 | 0.69 | 1.08 | 
********************
80th-quantile error in x, y, and z over all gazes
$\sigma_{\rm radius}$ [mm] | 0.0 | 0.2 | 0.5 | 1.0 | 3.0 | 
0.0 | 0.00 | 0.15 | 0.41 | 0.84 | 2.50 | 
0.3 | 2.60 | 2.77 | 2.86 | 3.29 | 3.33 | 
0.7 | 7.36 | 7.03 | 6.25 | 6.37 | 7.92 | 
********************
80th-quantile error in x and y for nominal gaze
$\sigma_{\rm radius}$ [mm] | 0.0 | 0.2 | 0.5 | 1.0 | 3.0 | 
0.0 | 0.00 | 0.04 | 0.10 | 0.14 | 0.43 | 
0.3 | 0.17 | 0.16 | 0.20 | 0.21 | 0.50 | 
0.7 | 0.53 | 0.50 | 0.